# Time Series Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA, ARMA, AR
from arch import arch_model

plt.rcParams['figure.figsize'] = (15.0, 5.0)

## Cryptocurrency Time Series

In [ ]:
df = pd.read_csv('../data/cryptos/bitcoin_price.csv')

df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
df = df.set_index('Date')
prices = df['Close']

prices_log = pd.Series(np.log(prices)).asfreq('1d')
crypto_returns = (prices_log - prices_log.shift()).dropna()
crypto_returns_abs = pd.Series(np.abs(crypto_returns))

In [ ]:
plt.title('Prices')
prices.plot()
plt.show()

plt.title('crypto_returns')
crypto_returns.plot()
plt.show()

plt.title('Absolute crypto_returns')
crypto_returns_abs.plot()
plt.show()

In [ ]:
result = adfuller(crypto_returns, autolag='AIC')
output = pd.Series(result[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
print("Augmented Dickey–Fuller test for crypto_returns:\n")
print(output)
print()

result = adfuller(crypto_returns_abs, autolag='AIC')
output = pd.Series(result[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
print('Augmented Dickey–Fuller test for Absolute crypto_returns:\n')
print(output)

## Currency Exchange Rates Time Series

In [ ]:
df = pd.read_csv('../data/currencies/EUR_USD Historical Data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
df = df.set_index('Date')

rates = df['Price']
rates = rates[crypto_returns.index]
rates.interpolate(inplace=True)  # Handle weekend missing data

rates_log = pd.Series(np.log(rates)).asfreq('1d')
rate_returns = (rates_log - rates_log.shift()).dropna()
rate_returns_abs = pd.Series(np.abs(rate_returns))

In [ ]:
plt.title('Exchange Rates')
rates.plot()
plt.show()

plt.title('crypto_returns')
rates_crypto_returns.plot()
plt.show()

plt.title('Absolute crypto_returns')
rates_crypto_returns_abs.plot()
plt.show()

In [ ]:
result = adfuller(rates_crypto_returns, autolag='AIC')
output = pd.Series(result[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
print("Augmented Dickey–Fuller test for rates_crypto_returns:\n")
print(output)
print()

result = adfuller(rates_crypto_returns_abs, autolag='AIC')
output = pd.Series(result[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
print('Augmented Dickey–Fuller test for Absolute rates_crypto_returns:\n')
print(output)

In [ ]:
def plot_acf(returns):
    lag_acf = acf(returns, nlags=20)
    lag_pacf = pacf(returns, nlags=20, method='ols')

    ######################### ACF ##########################################

    plt.subplot(121)
    plt.bar(x=range(len(lag_acf)), height=lag_acf, alpha=0.3, color='green')
    plt.plot(lag_acf)
    plt.axhline(y=0, linestyle='--',color='blue')
    plt.axhline(y=-1.96/np.sqrt(len(returns)), linestyle='--', color='pink')
    plt.axhline(y=1.96/np.sqrt(len(returns)), linestyle='--', color='blue')
    plt.title('Autocorrelation Function')

    ######################### PACF ##########################################

    plt.subplot(122)
    plt.bar(x=range(len(lag_pacf)), height=lag_pacf, alpha=0.3, color='green')
    plt.plot(lag_pacf)
    plt.axhline(y=0, linestyle='--',color='blue')
    plt.axhline(y=-1.96/np.sqrt(len(returns)), linestyle='--', color='pink')
    plt.axhline(y=1.96/np.sqrt(len(returns)), linestyle='--', color='blue')
    plt.title('Partial autocorrelation plot')
    plt.show()

## Cryptocurrency Analysis

In [ ]:
plot_acf(crypto_returns_abs)

In [ ]:
model = ARMA(crypto_returns_abs, order=(5, 2)).fit()
plt.plot(crypto_returns_abs)
plt.plot(model.fittedvalues, color='red')
plt.title('ARMA Mean Absolute Scaled Error: %.4f'% (sum(abs(model.resid) / crypto_returns_abs.mean()) / crypto_returns_abs.size))
plt.show()

print(model.summary())

In [ ]:
exog = rates[crypto_returns.index].dropna(inplace=True)
model = ARMA(crypto_returns_abs, order=(5, 2), exog=exog).fit()
plt.plot(crypto_returns_abs)
plt.plot(model.fittedvalues, color='red')
plt.title('VARMA Mean Absolute Scaled Error: %.4f'% (sum(abs(model.resid) / crypto_returns_abs.mean()) / crypto_returns_abs.size))
plt.show()

print(model.summary())

In [ ]:
model = arch_model(crypto_returns_abs, p=1, o=0, q=1).fit(update_freq=-1)
plt.plot(crypto_returns_abs)
plt.plot(model.conditional_volatility, color='red')
plt.title('ARCH Mean Absolute Scaled Error: %.4f'% (sum(abs(model.resid) / crypto_returns_abs.mean()) / crypto_returns_abs.size))
plt.show()

print(model.summary())

## Currency Analysis

In [ ]:
plot_acf(rate_returns_abs)

In [ ]:
model = ARMA(rate_returns_abs, order=(7, 2)).fit()
plt.plot(rate_returns_abs)
plt.plot(model.fittedvalues, color='red')
plt.title('ARMA Mean Absolute Scaled Error: %.4f'% (sum(abs(model.resid) / rate_returns_abs.mean()) / rate_returns_abs.size))
plt.show()

print(model.summary())

In [ ]:
model = arch_model(rate_returns_abs, p=4, o=0, q=3).fit(update_freq=-1)
plt.plot(rate_returns_abs)
plt.plot(model.conditional_volatility, color='red')
plt.title('GARCH Mean Absolute Scaled Error: %.4f'% (sum(abs(model.resid) / rate_returns_abs.mean()) / rate_returns_abs.size))
plt.show()

print(model.summary())